In [1]:
# Use the Training data and create the model and data drift detector object and save it.

# Then create a schedule which runs after every specific period that checks the data from the logging table for drift and raises a trigger

# If trigger happens then all the data till 1 week before should be queried from the logging table again
# and model should be retrained. Then the new model & old model should test on the last 1 week data for selecting the best.

# Once the final model is selected, the old model pickle file should be pushed to archive and the new model should take its position along with all the other features and mappings


In [2]:
%%capture
!pip install alibi

In [3]:
%%capture
!pip install alibi-detect

In [5]:
%%capture

!pip install snowflake-connector-python

In [6]:
%%capture

!pip install snowflake-sqlalchemy

In [7]:
%%capture

!pip install xgboost

In [1]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

import scipy.stats as stats
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle 

import sklearn
from sklearn import metrics

import alibi 
from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.saving import save_detector, load_detector

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost
from datetime import datetime, timedelta
import time  
import pytz    
tz_NY = pytz.timezone('Asia/Kolkata')

import snowflake_creds

import warnings
warnings.filterwarnings('ignore')

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="cr21746.ap-south-1",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

## Creating the Model and Data drift detector object from Training set:

#### Data Drift detector:

In [3]:
query = """

SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.HEALTHSCHEMA.HEALTH_DATA

"""

In [4]:
# Loading the train data
with engine.connect() as conn:
    df_train = pd.DataFrame(pd.read_sql(query,conn))
    df_train.columns = [col.upper() for col in df_train.columns.tolist()]

In [5]:
print(df_train.shape)
df_train.head(3)

(236704, 19)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT,AGE,ADMISSION_DEPOSIT,ADMISSION_DATE,DISCHARGE_DATE
0,1,8,c,3,Z,3,radiotherapy,R,F,2,31397,7,Emergency,Extreme,2,51-60,4911,2022-08-10,2022-08-20
1,3,10,e,1,X,2,anesthesia,S,E,2,31397,7,Trauma,Extreme,2,51-60,4745,2022-09-09,2022-10-19
2,4,26,b,2,Y,2,radiotherapy,R,D,2,31397,7,Trauma,Extreme,2,51-60,7272,2022-09-22,2022-11-11


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236704 entries, 0 to 236703
Data columns (total 19 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   CASE_ID                            236704 non-null  int64 
 1   HOSPITAL_CODE                      236704 non-null  int64 
 2   HOSPITAL_TYPE_CODE                 236704 non-null  object
 3   CITY_CODE_HOSPITAL                 236704 non-null  int64 
 4   HOSPITAL_REGION_CODE               236704 non-null  object
 5   AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL  236704 non-null  int64 
 6   DEPARTMENT                         236704 non-null  object
 7   WARD_TYPE                          236704 non-null  object
 8   WARD_FACILITY_CODE                 236704 non-null  object
 9   BED_GRADE                          236704 non-null  int64 
 10  PATIENTID                          236704 non-null  int64 
 11  CITY_CODE_PATIENT                  236704 non-null  

In [7]:
# Getting the numerical and categorical columns for creating the datadrift object
num_columns = ['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL','VISITORS_WITH_PATIENT','ADMISSION_DEPOSIT']
id_columns = ['CASE_ID','PATIENTID','ADMISSION_DATE','DISCHARGE_DATE']
cat_columns = [col for col in df_train.columns.tolist() if col not in num_columns+id_columns]

In [8]:
num_columns

['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL',
 'VISITORS_WITH_PATIENT',
 'ADMISSION_DEPOSIT']

In [9]:
cat_columns

['HOSPITAL_CODE',
 'HOSPITAL_TYPE_CODE',
 'CITY_CODE_HOSPITAL',
 'HOSPITAL_REGION_CODE',
 'DEPARTMENT',
 'WARD_TYPE',
 'WARD_FACILITY_CODE',
 'BED_GRADE',
 'CITY_CODE_PATIENT',
 'TYPE_OF_ADMISSION',
 'SEVERITY_OF_ILLNESS',
 'AGE']

In [10]:
X_train = df_train[num_columns + cat_columns]
print(X_train.shape)
X_train.head()

(236704, 15)


,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,VISITORS_WITH_PATIENT,ADMISSION_DEPOSIT,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,AGE
0,3,2,4911,8,c,3,Z,radiotherapy,R,F,2,7,Emergency,Extreme,51-60
1,2,2,4745,10,e,1,X,anesthesia,S,E,2,7,Trauma,Extreme,51-60
2,2,2,7272,26,b,2,Y,radiotherapy,R,D,2,7,Trauma,Extreme,51-60
3,2,2,5558,26,b,2,Y,radiotherapy,S,D,2,7,Trauma,Extreme,51-60
4,2,2,4449,23,a,6,X,anesthesia,S,F,2,7,Trauma,Extreme,51-60


In [11]:
cat_indices = np.arange(3,15)
cat_indices

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [12]:
# category dict for the driftdetector to identify unique categories
categories_per_feature = {f: None for f in cat_indices}
categories_per_feature

{3: None,
 4: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: None,
 11: None,
 12: None,
 13: None,
 14: None}

In [13]:
# Initialize the detector
cd = TabularDrift(X_train.values, p_val=.05, categories_per_feature=categories_per_feature)

In [14]:
cd

TabularDrift

In [15]:
cd.get_config()

{'name': 'TabularDrift',
 'meta': {'version': '0.11.1'},
 'x_ref': array([[3, 2, 4911, ..., 'Emergency', 'Extreme', '51-60'],
        [2, 2, 4745, ..., 'Trauma', 'Extreme', '51-60'],
        [2, 2, 7272, ..., 'Trauma', 'Extreme', '51-60'],
        ...,
        [3, 15, 4241, ..., 'Emergency', 'Moderate', '41-50'],
        [4, 4, 3036, ..., 'Emergency', 'Extreme', '41-50'],
        [4, 4, 4326, ..., 'Trauma', 'Extreme', '31-40']], dtype=object),
 'p_val': 0.05,
 'categories_per_feature': {3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None,
  9: None,
  10: None,
  11: None,
  12: None,
  13: None,
  14: None},
 'x_ref_preprocessed': False,
 'preprocess_at_init': True,
 'update_x_ref': None,
 'preprocess_fn': None,
 'correction': 'bonferroni',
 'alternative': 'two-sided',
 'n_features': None,
 'input_shape': None,
 'data_type': None}

In [16]:
# # Using pickle to save and load it the trained detector
# with open('Trained_Drift_Detector.pkl','wb') as F:
#     pickle.dump(cd,F)

with open('Trained_Drift_Detector.pkl','rb') as F:
    trained_drift_model = pickle.load(F)    

In [17]:
trained_drift_model.get_config()

{'name': 'TabularDrift',
 'meta': {'version': '0.11.1'},
 'x_ref': array([[3, 2, 4911, ..., 'Emergency', 'Extreme', '51-60'],
        [2, 2, 4745, ..., 'Trauma', 'Extreme', '51-60'],
        [2, 2, 7272, ..., 'Trauma', 'Extreme', '51-60'],
        ...,
        [3, 15, 4241, ..., 'Emergency', 'Moderate', '41-50'],
        [4, 4, 3036, ..., 'Emergency', 'Extreme', '41-50'],
        [4, 4, 4326, ..., 'Trauma', 'Extreme', '31-40']], dtype=object),
 'p_val': 0.05,
 'categories_per_feature': {3: None,
  4: None,
  5: None,
  6: None,
  7: None,
  8: None,
  9: None,
  10: None,
  11: None,
  12: None,
  13: None,
  14: None},
 'x_ref_preprocessed': False,
 'preprocess_at_init': True,
 'update_x_ref': None,
 'preprocess_fn': None,
 'correction': 'bonferroni',
 'alternative': 'two-sided',
 'n_features': None,
 'input_shape': None,
 'data_type': None}

In [18]:
preds = trained_drift_model.predict(X_train.values)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? No!


In [19]:
# If you are interested in individual feature-wise drift, this is also possible:
fpreds = trained_drift_model.predict(X_train.values, drift_type='feature')
fpreds

{'data': {'is_drift': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'distance': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        dtype=float32),
  'p_val': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        dtype=float32),
  'threshold': 0.05},
 'meta': {'name': 'TabularDrift',
  'online': False,
  'data_type': None,
  'version': '0.11.1',
  'detector_type': 'drift'}}

In [20]:
for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = X_train.columns.tolist()[f]
    # print(f, fname)
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL -- Drift? No! -- K-S 0.000 -- p-value 1.000
VISITORS_WITH_PATIENT -- Drift? No! -- K-S 0.000 -- p-value 1.000
ADMISSION_DEPOSIT -- Drift? No! -- K-S 0.000 -- p-value 1.000
HOSPITAL_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
HOSPITAL_TYPE_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
CITY_CODE_HOSPITAL -- Drift? No! -- Chi2 0.000 -- p-value 1.000
HOSPITAL_REGION_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
DEPARTMENT -- Drift? No! -- Chi2 0.000 -- p-value 1.000
WARD_TYPE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
WARD_FACILITY_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
BED_GRADE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
CITY_CODE_PATIENT -- Drift? No! -- Chi2 0.000 -- p-value 1.000
TYPE_OF_ADMISSION -- Drift? No! -- Chi2 0.000 -- p-value 1.000
SEVERITY_OF_ILLNESS -- Drift? No! -- Chi2 0.000 -- p-value 1.000
AGE -- Drift? No! -- Chi2 0.000 -- p-value 1.000


In [21]:
temp = pd.DataFrame()
temp['Time Period'] = [str('2023-01-01') + ' to ' + str('2023-01-07')]*len(X_train.columns.tolist())
temp['Features'] = X_train.columns.tolist()
temp['Is Drift'] = fpreds['data']['is_drift']
temp['Stat Test'] = temp['Features'].apply(lambda x: 'Chi2' if x in cat_columns else 'K-S')
temp['Stats Value'] = fpreds['data']['distance']
temp['P-value'] = fpreds['data']['p_val']
print(temp.shape)
temp

(15, 6)


,Time Period,Features,Is Drift,Stat Test,Stats Value,P-value
0,2023-01-01 to 2023-01-07,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0,K-S,0.0,1.0
1,2023-01-01 to 2023-01-07,VISITORS_WITH_PATIENT,0,K-S,0.0,1.0
2,2023-01-01 to 2023-01-07,ADMISSION_DEPOSIT,0,K-S,0.0,1.0
3,2023-01-01 to 2023-01-07,HOSPITAL_CODE,0,Chi2,0.0,1.0
4,2023-01-01 to 2023-01-07,HOSPITAL_TYPE_CODE,0,Chi2,0.0,1.0
5,2023-01-01 to 2023-01-07,CITY_CODE_HOSPITAL,0,Chi2,0.0,1.0
6,2023-01-01 to 2023-01-07,HOSPITAL_REGION_CODE,0,Chi2,0.0,1.0
7,2023-01-01 to 2023-01-07,DEPARTMENT,0,Chi2,0.0,1.0
8,2023-01-01 to 2023-01-07,WARD_TYPE,0,Chi2,0.0,1.0
9,2023-01-01 to 2023-01-07,WARD_FACILITY_CODE,0,Chi2,0.0,1.0


##### Creating a noise in the train data to check if drift detector is working

In [22]:
temp = X_train.copy()
temp.loc[:5,'HOSPITAL_CODE'] = 100
temp.head()

,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,VISITORS_WITH_PATIENT,ADMISSION_DEPOSIT,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,AGE
0,3,2,4911,100,c,3,Z,radiotherapy,R,F,2,7,Emergency,Extreme,51-60
1,2,2,4745,100,e,1,X,anesthesia,S,E,2,7,Trauma,Extreme,51-60
2,2,2,7272,100,b,2,Y,radiotherapy,R,D,2,7,Trauma,Extreme,51-60
3,2,2,5558,100,b,2,Y,radiotherapy,S,D,2,7,Trauma,Extreme,51-60
4,2,2,4449,100,a,6,X,anesthesia,S,F,2,7,Trauma,Extreme,51-60


In [23]:
# If you are interested in individual feature-wise drift, this is also possible:
fpreds = trained_drift_model.predict(temp.values, drift_type='feature')
fpreds

{'data': {'is_drift': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'distance': array([0.       , 0.       , 0.       , 6.0004225, 0.       , 0.       ,
         0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
         0.       , 0.       , 0.       ], dtype=float32),
  'p_val': array([1.       , 1.       , 1.       , 0.9999999, 1.       , 1.       ,
         1.       , 1.       , 1.       , 1.       , 1.       , 1.       ,
         1.       , 1.       , 1.       ], dtype=float32),
  'threshold': 0.05},
 'meta': {'name': 'TabularDrift',
  'online': False,
  'data_type': None,
  'version': '0.11.1',
  'detector_type': 'drift'}}

In [24]:
for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = temp.columns.tolist()[f]
    # print(f, fname)
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL -- Drift? No! -- K-S 0.000 -- p-value 1.000
VISITORS_WITH_PATIENT -- Drift? No! -- K-S 0.000 -- p-value 1.000
ADMISSION_DEPOSIT -- Drift? No! -- K-S 0.000 -- p-value 1.000
HOSPITAL_CODE -- Drift? No! -- Chi2 6.000 -- p-value 1.000
HOSPITAL_TYPE_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
CITY_CODE_HOSPITAL -- Drift? No! -- Chi2 0.000 -- p-value 1.000
HOSPITAL_REGION_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
DEPARTMENT -- Drift? No! -- Chi2 0.000 -- p-value 1.000
WARD_TYPE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
WARD_FACILITY_CODE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
BED_GRADE -- Drift? No! -- Chi2 0.000 -- p-value 1.000
CITY_CODE_PATIENT -- Drift? No! -- Chi2 0.000 -- p-value 1.000
TYPE_OF_ADMISSION -- Drift? No! -- Chi2 0.000 -- p-value 1.000
SEVERITY_OF_ILLNESS -- Drift? No! -- Chi2 0.000 -- p-value 1.000
AGE -- Drift? No! -- Chi2 0.000 -- p-value 1.000


#### Data Drift Scoring:

In [25]:
def data_monitoring_batch_query(a):
    query = f"""

        SELECT CASE_ID,
               COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
               COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
               COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
               COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
               COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
               COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
               COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
               COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
               COALESCE(BED_GRADE,0) AS BED_GRADE,
               PATIENTID,
               COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
               COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
               COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
               COALESCE(VISITORS_WITH_PATIENT_X,0) AS VISITORS_WITH_PATIENT,
               COALESCE(AGE,'None') AS AGE,
               COALESCE(ADMISSION_DEPOSIT_X,0) AS ADMISSION_DEPOSIT,
               ADMISSION_DATE,
               DISCHARGE_DATE

            FROM HEALTHDB.HEALTHSCHEMA.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
            WHERE ADMISSION_DATE >= CURRENT_DATE-144+{a*7} AND ADMISSION_DATE < CURRENT_DATE-144+{(a+1)*7}        

        """
    return query

In [26]:
# print(data_monitoring_batch_query(0))

In [27]:
def data_monitoring(batch_id):
    # Loading the train data
    with engine.connect() as conn:
        batch_df = pd.DataFrame(pd.read_sql(data_monitoring_batch_query(batch_id),conn))
        batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]
    
    # Getting the numerical and categorical columns for creating the datadrift object
    num_columns = ['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL','VISITORS_WITH_PATIENT','ADMISSION_DEPOSIT']
    id_columns = ['CASE_ID','PATIENTID','ADMISSION_DATE','DISCHARGE_DATE']
    cat_columns = [col for col in batch_df.columns.tolist() if col not in num_columns+id_columns]
    
    # Getting the final prepared data
    batch_final = batch_df[num_columns + cat_columns]
    
    # Loading the Trained data drift detector
    with open('Trained_Drift_Detector.pkl','rb') as F:
        trained_drift_model = pickle.load(F)    
    
    # Checking for drift
    # If you are interested in individual feature-wise drift, this is also possible:
    fpreds = trained_drift_model.predict(batch_final.values, drift_type='feature')
    
    log_df = pd.DataFrame()
    log_df['Time Period'] = ([str(batch_df['ADMISSION_DATE'].min()) + ' to ' + 
                              str(batch_df['ADMISSION_DATE'].max())]
                              * len(batch_final.columns.tolist())
                            )
    log_df['Total Records'] = batch_df.shape[0]
    log_df['Features'] = batch_final.columns.tolist()
    log_df['Is Drift'] = fpreds['data']['is_drift']
    log_df['Stat Test'] = log_df['Features'].apply(lambda x: 'Chi2' if x in cat_columns else 'K-S')
    log_df['Stats Value'] = np.round(fpreds['data']['distance'])
    log_df['P-value'] = np.round(fpreds['data']['p_val'])
    
    return log_df

In [28]:
data_monitoring(1)

,Time Period,Total Records,Features,Is Drift,Stat Test,Stats Value,P-value
0,2022-12-12 to 2022-12-18,16663,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,1,K-S,0.0,0.0
1,2022-12-12 to 2022-12-18,16663,VISITORS_WITH_PATIENT,1,K-S,0.0,0.0
2,2022-12-12 to 2022-12-18,16663,ADMISSION_DEPOSIT,1,K-S,0.0,0.0
3,2022-12-12 to 2022-12-18,16663,HOSPITAL_CODE,1,Chi2,102.0,0.0
4,2022-12-12 to 2022-12-18,16663,HOSPITAL_TYPE_CODE,1,Chi2,14.0,0.0
5,2022-12-12 to 2022-12-18,16663,CITY_CODE_HOSPITAL,1,Chi2,33.0,0.0
6,2022-12-12 to 2022-12-18,16663,HOSPITAL_REGION_CODE,1,Chi2,8.0,0.0
7,2022-12-12 to 2022-12-18,16663,DEPARTMENT,1,Chi2,32.0,0.0
8,2022-12-12 to 2022-12-18,16663,WARD_TYPE,1,Chi2,49.0,0.0
9,2022-12-12 to 2022-12-18,16663,WARD_FACILITY_CODE,1,Chi2,12.0,0.0


In [29]:
t = data_monitoring(0)
t

,Time Period,Total Records,Features,Is Drift,Stat Test,Stats Value,P-value
0,2022-12-05 to 2022-12-11,16737,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,1,K-S,0.0,0.0
1,2022-12-05 to 2022-12-11,16737,VISITORS_WITH_PATIENT,1,K-S,0.0,0.0
2,2022-12-05 to 2022-12-11,16737,ADMISSION_DEPOSIT,1,K-S,0.0,0.0
3,2022-12-05 to 2022-12-11,16737,HOSPITAL_CODE,1,Chi2,120.0,0.0
4,2022-12-05 to 2022-12-11,16737,HOSPITAL_TYPE_CODE,1,Chi2,27.0,0.0
5,2022-12-05 to 2022-12-11,16737,CITY_CODE_HOSPITAL,1,Chi2,44.0,0.0
6,2022-12-05 to 2022-12-11,16737,HOSPITAL_REGION_CODE,0,Chi2,1.0,1.0
7,2022-12-05 to 2022-12-11,16737,DEPARTMENT,1,Chi2,17.0,0.0
8,2022-12-05 to 2022-12-11,16737,WARD_TYPE,1,Chi2,29.0,0.0
9,2022-12-05 to 2022-12-11,16737,WARD_FACILITY_CODE,0,Chi2,10.0,0.0


In [30]:
print(t.shape[0])
print(t['Is Drift'].sum())

15
13


### Model Drift detector:

In [53]:
# Get the data from the logging table in batches (7 days)
# Using the predicted and actual LOS calculate the performance metrics dict
# Then use the ref_metric_dict (from training) to compare with the current_metric_dict for model drift

In [31]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="cr21746.ap-south-1",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

In [32]:
# Function to check model drift
def check_model_drift(ref_metric_dict,cur_metric_dict,type='classification',tol=0.1):
    if type == 'classification':
        precision_change = abs((cur_metric_dict['Precision']-ref_metric_dict['Precision'])/ref_metric_dict['Precision'])
        recall_change = abs((cur_metric_dict['Recall']-ref_metric_dict['Recall'])/ref_metric_dict['Recall'])
        roc_auc_change = abs((cur_metric_dict['Roc-Auc']-ref_metric_dict['Roc-Auc'])/ref_metric_dict['Roc-Auc'])

        counter = 0
        for i in [precision_change,recall_change,roc_auc_change]:
            if i > tol:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            print("Change in Precision: "+ str(np.round(100*precision_change,2))+"%")
            print("Change in Recall: "+ str(np.round(100*recall_change,2))+"%")
            print("Change in Roc-Auc: "+ str(np.round(100*roc_auc_change,2))+"%")
            return 1
        else:
            print("There is no model drift.")
            return 0

    elif type == 'regression':
        rmse_change = abs((cur_metric_dict['RMSE']-ref_metric_dict['RMSE'])/ref_metric_dict['RMSE'])
        mae_change = abs((cur_metric_dict['MAE']-ref_metric_dict['MAE'])/ref_metric_dict['MAE'])
        
        counter = 0
        for i in [rmse_change,mae_change]:
            if i > tol:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            RMSE_CHANGE = np.round(100*rmse_change,2)
            MAE_CHANGE = np.round(100*mae_change,2)
            print("Change in RMSE: "+ str(np.round(100*rmse_change,2))+"%")
            print("Change in MAE: "+ str(np.round(100*mae_change,2))+"%")
            return 1, RMSE_CHANGE, MAE_CHANGE
        else:
            print("There is no model drift.")
            RMSE_CHANGE = 'NONE'
            MAE_CHANGE = 'NONE'
            return 0, RMSE_CHANGE, MAE_CHANGE
        
        

In [33]:
def model_monitoring_batch_query(a):
    query_sim = f"""

        SELECT *
        FROM TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
        WHERE ADMISSION_DATE >= CURRENT_DATE-144+{a*7} AND ADMISSION_DATE < CURRENT_DATE-144+{(a+1)*7}
        
    """
    return query_sim

In [37]:
print(model_monitoring_batch_query(2))



        SELECT *
        FROM TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
        WHERE ADMISSION_DATE >= CURRENT_DATE-144+14 AND ADMISSION_DATE < CURRENT_DATE-144+21
        
    


In [38]:
# Loading the train data
with engine.connect() as conn:
    batch_df = pd.DataFrame(pd.read_sql(model_monitoring_batch_query(0),conn))
    batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]

In [39]:
print(batch_df.shape)
batch_df.head()

(16737, 54)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT_X,AGE,ADMISSION_DEPOSIT_X,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS_X,TYPE_OF_ADMISSION_TRAUMA,ADMISSION_DAY_THU,ADMISSION_DAY_SAT,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_Y,ADMISSION_DAY_TUE,SEVERITY_OF_ILLNESS_MINOR,BED_GRADE_2,ADMISSION_MONTH_OCT,VISITORS_WITH_PATIENT_Y,CITY_CODE_HOSPITAL_7,DEPARTMENT_ILLNESS_GYNECOLOGY-EXTREME,AGE_31-40,AGE_41-50,ADMISSION_MONTH_NOV,TYPE_OF_ADMISSION_EMERGENCY,ADMISSION_DEPOSIT_Y,ADMISSION_DAY_WED,ILLNESS_BEDGRADE_EXTREME-1,AGE_71-80,WARD_TYPE_S,AGE_51-60,WARD_TYPE_Q,WARD_TYPE_P,ADMISSION_DAY_FRI,ADMISSION_DAY_SUN,ADMISSION_DAY_MON,CITY_CODE_PATIENT_8,LOS_Y,PREDICTED_LOS
0,9595,28,b,11,X,3,anesthesia,R,F,3,37254,1,Emergency,Minor,5,61-70,7611,2022-12-05,2023-01-14,Dec,Mon,Emergency-Minor,Minor-3,anesthesia-Minor,40,0,0,0,3,0,1,0,0,5,0,0,0,0,0,1,7611,0,0,0,0,0,0,0,0,0,1,0,40,48.0
1,9596,28,b,11,X,4,gynecology,R,F,4,91007,1,Emergency,Moderate,4,81-90,3500,2022-12-05,2023-01-14,Dec,Mon,Emergency-Moderate,Moderate-4,gynecology-Moderate,40,0,0,0,4,0,0,0,0,4,0,0,0,0,0,1,3500,0,0,0,0,0,0,0,0,0,1,0,40,43.0
2,9597,17,e,1,X,2,anesthesia,R,E,3,2352,8,Urgent,Minor,4,31-40,4282,2022-12-05,2022-12-25,Dec,Mon,Urgent-Minor,Minor-3,anesthesia-Minor,20,0,0,0,2,0,1,0,0,4,0,0,1,0,0,0,4282,0,0,0,0,0,0,0,0,0,1,1,20,53.0
3,9598,14,a,1,X,2,radiotherapy,Q,E,3,122110,0,Emergency,Minor,2,71-80,9096,2022-12-05,2023-02-03,Dec,Mon,Emergency-Minor,Minor-3,radiotherapy-Minor,60,0,0,0,2,0,1,0,0,2,0,0,0,0,0,1,9096,0,0,1,0,0,1,0,0,0,1,0,60,41.0
4,9599,9,d,5,Z,2,gynecology,R,F,3,123339,5,Emergency,Moderate,2,51-60,4557,2022-12-05,2022-12-15,Dec,Mon,Emergency-Moderate,Moderate-3,gynecology-Moderate,10,0,0,0,2,0,0,0,0,2,0,0,0,0,0,1,4557,0,0,0,0,1,0,0,0,0,1,0,10,25.0


In [45]:
# Creating the current performance dict (from scoring)

actual = batch_df['LOS_X']
predicted = batch_df['PREDICTED_LOS']

rmse = np.sqrt(metrics.mean_squared_error(actual,predicted))
mae = np.sqrt(metrics.mean_absolute_error(actual,predicted))
print("RMSE: ", rmse)
print("MAE: ", mae)

scoring_ref_metrics = {}
scoring_ref_metrics['RMSE'] = rmse
scoring_ref_metrics['MAE'] = mae #+ 0.2*mae
print(scoring_ref_metrics)

RMSE:  14.908764378801582
MAE:  3.3061688138399608
{'RMSE': 14.908764378801582, 'MAE': 3.3061688138399608}


In [46]:
# Loading the reference performance dict (from training)

with open('MODEL_XGB_PERFM_METRICS.pkl', 'rb') as F:
    model_ref_metric = pickle.load(F)

model_ref_metric

{'RMSE': 13.607345304032433, 'MAE': 3.1702232476032353}

In [47]:
check_model_drift(model_ref_metric,scoring_ref_metrics,type='regression',tol=0.1)

There is no model drift.


(0, 'NONE', 'NONE')

In [48]:
def model_monitoring(batch_id):
    # Loading the train data
    with engine.connect() as conn:
        batch_df = pd.DataFrame(pd.read_sql(model_monitoring_batch_query(batch_id),conn))
        batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]
    
#     print(batch_df.shape)
    
    # Creating the current performance dict (from scoring)
    actual = batch_df['LOS_X']
    predicted = batch_df['PREDICTED_LOS']

    rmse = np.sqrt(metrics.mean_squared_error(actual,predicted))
    mae = np.sqrt(metrics.mean_absolute_error(actual,predicted))
#     print("RMSE: ", rmse)
#     print("MAE: ", mae)

    scoring_ref_metrics = {}
    scoring_ref_metrics['RMSE'] = rmse
    scoring_ref_metrics['MAE'] = mae #+ 0.2*mae
#     print(scoring_ref_metrics)
    
    
    # Loading the reference performance dict (from training)
    with open('MODEL_XGB_PERFM_METRICS.pkl', 'rb') as F:
        model_ref_metric = pickle.load(F)
        
#     print(model_ref_metric)
    
    # Check for model drift
    model_drift, RMSE_CHANGE, MAE_CHANGE = check_model_drift(model_ref_metric,scoring_ref_metrics,type='regression',tol=0.1)
    
    # Log values
    log = {}
    log['Time Period'] = str(batch_df['ADMISSION_DATE'].min()) + ' to ' + str(batch_df['ADMISSION_DATE'].max())
    log['Total Records'] = batch_df.shape[0]
    log['Scoring Metrics'] = scoring_ref_metrics
    log['Training Metrics'] = model_ref_metric
    log['Model Drift IND'] = model_drift
    log['RMSE Change'] = RMSE_CHANGE
    log['MAE Change'] = MAE_CHANGE
    
    return log
    

In [52]:
model_monitoring(3)

There is no model drift.


{'Time Period': '2022-12-26 to 2022-12-30',
 'Total Records': 12048,
 'Scoring Metrics': {'RMSE': 14.882318358857793, 'MAE': 3.2823626220255004},
 'Training Metrics': {'RMSE': 13.607345304032433, 'MAE': 3.1702232476032353},
 'Model Drift IND': 0,
 'RMSE Change': 'NONE',
 'MAE Change': 'NONE'}

In [ ]:
# Then create a dictionary with has two keys --> {data_drift:'Yes', model_drift:'Yes'}
# Based on the logic, it should call the retraining function 
# Once above steps are completed, create the retraining script that takes in all the data from the training and logging table (except the last 1 or 2 week)

# Model Monitoring & Retraining Pipeline:

In [53]:
data_log_df = data_monitoring(0)
model_log_dict = model_monitoring(0)

There is no model drift.


In [54]:
data_log_df

,Time Period,Total Records,Features,Is Drift,Stat Test,Stats Value,P-value
0,2022-12-05 to 2022-12-11,16737,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,1,K-S,0.0,0.0
1,2022-12-05 to 2022-12-11,16737,VISITORS_WITH_PATIENT,1,K-S,0.0,0.0
2,2022-12-05 to 2022-12-11,16737,ADMISSION_DEPOSIT,1,K-S,0.0,0.0
3,2022-12-05 to 2022-12-11,16737,HOSPITAL_CODE,1,Chi2,120.0,0.0
4,2022-12-05 to 2022-12-11,16737,HOSPITAL_TYPE_CODE,1,Chi2,27.0,0.0
5,2022-12-05 to 2022-12-11,16737,CITY_CODE_HOSPITAL,1,Chi2,44.0,0.0
6,2022-12-05 to 2022-12-11,16737,HOSPITAL_REGION_CODE,0,Chi2,1.0,1.0
7,2022-12-05 to 2022-12-11,16737,DEPARTMENT,1,Chi2,17.0,0.0
8,2022-12-05 to 2022-12-11,16737,WARD_TYPE,1,Chi2,29.0,0.0
9,2022-12-05 to 2022-12-11,16737,WARD_FACILITY_CODE,0,Chi2,10.0,0.0


In [57]:
# Data drift condition
data_log_df['Is Drift'].sum() > 0

True

In [55]:
model_log_dict

{'Time Period': '2022-12-05 to 2022-12-11',
 'Total Records': 16737,
 'Scoring Metrics': {'RMSE': 14.908764378801582, 'MAE': 3.3061688138399608},
 'Training Metrics': {'RMSE': 13.607345304032433, 'MAE': 3.1702232476032353},
 'Model Drift IND': 0,
 'RMSE Change': 'NONE',
 'MAE Change': 'NONE'}

In [58]:
# Model drift condition
model_log_dict['Model Drift IND']

0

In [61]:
# Max date for retraining 
max_date = model_log_dict['Time Period'].split(' ')[2]
max_date

'2022-12-11'